# Rock, Paper, and Scissors

## i'm running it in Google Collab


In [1]:
from os import getcwd

import tensorflow as tf
import tensorflow_datasets as tfds

## Make one hot encoding for the labels. the labels have 3 classes


In [2]:
def OneHotEncoding(feature, label):
    one_hot = tf.one_hot(label, 3)
    return feature, one_hot


# Test the function
_,one_hot = OneHotEncoding(["a", "b", "c", "a", "b"], [1, 2, 3, 1, 2])
print(one_hot)

tf.Tensor(
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(5, 3), dtype=float32)


## Load The Dataset and Explore it


In [3]:
filepath = f"{getcwd()}/data"

train_data = tfds.load("rock_paper_scissors:3.*.*", split="train", data_dir = filepath, as_supervised=True)
test_data = tfds.load("rock_paper_scissors:3.*.*", split="test", data_dir = filepath, as_supervised=True)

# Knowing the amount of data
print(f"Amount of train data: {len(list(train_data))}")
print(f"Amount of test data: {len(list(test_data))}")

# Knowing the shape of image
_,info = tfds.load("rock_paper_scissors:3.*.*", with_info=True, data_dir=filepath)

# DO NOT EDIT THIS
print(info.features['image'].shape)

Amount of train data: 2520
Amount of test data: 372
(300, 300, 3)


## Mapping dataset use one hot encode function


In [4]:
train_data = train_data.map(OneHotEncoding)
test_data = test_data.map(OneHotEncoding)

print(type(train_data))

<class 'tensorflow.python.data.ops.map_op._MapDataset'>


## Make a model CNN


In [5]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(32,(3, 3), activation="relu", input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64,(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128,(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(256,(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 73, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 35, 35, 128)       0

## Evaluate model


In [6]:
train_batches = train_data.shuffle(100).batch(10)
validation_batches = test_data.batch(32)

In [7]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get("accuracy") > 0.96 and logs.get("val_accuracy") > 0.96:
            print("\nReached 97% accuracy so cancelling training!")
            self.model.stop_training = True

# Buat instance dari callback
my_callback = MyCallback()

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
    metrics=["accuracy"],
)

history = model.fit(
    train_batches, epochs=1000, validation_data=validation_batches,validation_steps=1, callbacks=[my_callback]
)

print("Final Training Accuracy:-", history.history["accuracy"][-1])
print("Final Validation Accuracy:-", history.history["val_accuracy"][-1])

Epoch 1/1000
  5/252 [..............................] - ETA: 8s - loss: 25.6183 - accuracy: 0.3600

252/252 [==============================] - 16s 39ms/step - loss: 4.0982 - accuracy: 0.5623 - val_loss: 0.6843 - val_accuracy: 0.6875
Epoch 2/1000
252/252 [==============================] - 9s 35ms/step - loss: 0.5798 - accuracy: 0.7393 - val_loss: 0.6544 - val_accuracy: 0.7812
Epoch 3/1000
252/252 [==============================] - 9s 35ms/step - loss: 0.4030 - accuracy: 0.8206 - val_loss: 0.4978 - val_accuracy: 0.8750
Epoch 4/1000
252/252 [==============================] - 9s 35ms/step - loss: 0.3027 - accuracy: 0.8726 - val_loss: 0.4639 - val_accuracy: 0.8125
Epoch 5/1000
252/252 [==============================] - 9s 35ms/step - loss: 0.2058 - accuracy: 0.9202 - val_loss: 0.3092 - val_accuracy: 0.8750
Epoch 6/1000
252/252 [==============================] - 9s 34ms/step - loss: 0.1651 - accuracy: 0.9361 - val_loss: 0.4180 - val_accuracy: 0.8438
Epoch 7/1000
252/252 [==============================] - 9s 34ms/step - loss: 0.1115 - accuracy: 0.9591 - val_loss: 0.4514 - val_accuracy: 0.84